<a href="https://colab.research.google.com/github/rim-yu/procam-calibration/blob/master/procam_calibration_200605.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

%xmode Verbose
%pdb on

Exception reporting mode: Verbose
Automatic pdb calling has been turned ON


In [ ]:
import os
import os.path
from pathlib import Path
#cf: https://treyhunner.com/2018/12/why-you-should-be-using-pathlib/
import glob
import argparse
import cv2
import numpy as np
import json
from matplotlib import pyplot as plt 

In [ ]:
def main(proj_height, proj_width, chess_rows, chess_cols, chess_block_size,
    graycode_step, black_thr, white_thr,  camera):
  
# 2160 - 3840 - 10 - 7 - 68 - 1 - 40 - 5 - ""

    proj_shape = (proj_height, proj_width) # (2160, 3840)
    chess_shape = (chess_rows, chess_cols) # (10, 7)
    chess_block_size = chess_block_size # chess_block_size = 68 mm
    gc_step = graycode_step # gc_step = 1
    black_thr = black_thr # 40
    white_thr = white_thr # 5
    camera_param_file = camera # ""

    dirnames = sorted(glob.glob('./capture_*'))
    if len(dirnames) == 0: # capture_* 형식으로 된 폴더가 없을 경우 error 뱉음. 
        print('Directories \'./capture_*\' were not found')
        return

    print('Searching input files ...') # capture_* 형식으로 된 폴더가 있을 경우 파일들을 찾음. 
    used_dirnames = [] 
    gc_fname_lists = [] # 파일 이름 리스트. 
    for dname in dirnames: # capture_* 폴더 돌림. * = 1->4
        #gc_fnames = sorted(glob.glob(dname + '/graycode_*')) # graycode_* 파일들 가져옴. 
        gc_fnames = sorted(glob.glob(dname + '/graycode*'))
        if len(gc_fnames) == 0:  
            continue
        used_dirnames.append(dname) # capture_* 폴더가 순차적으로 붙음. 
        gc_fname_lists.append(gc_fnames) # 파일 이름 리스트. graycode_*가 붙음. * = 00->49
        print(' \'' + dname + '\' was found') # 폴더 찾았당. 

    camP = None # ?
    cam_dist = None # ? 
    path, ext = os.path.splitext(camera_param_file) # camera_param_file = "", 파일 이름을 확장자로부터 쪼개줌. 
    if(ext == ".json"): # 확장자명. 
        camP, cam_dist = loadCameraParam(camera_param_file) # 카메라 파라미터 저장된 json 파일 가져옴. 
        print('load camera parameters')
        print(camP)
        print(cam_dist)

    calibrate(used_dirnames, gc_fname_lists,
              proj_shape, chess_shape, chess_block_size, gc_step, black_thr, white_thr,
               camP, cam_dist) 
    # calibrate 함수 쓰는데, capture_* 폴더명들, graycode_* 파일명들, 그외 각종 변수들 + camP, cam_dist 

In [ ]:
def printNumpyWithIndent(tar, indentchar):
    print(indentchar + str(tar).replace('\n', '\n' + indentchar))  

In [ ]:
def loadCameraParam(json_file):
    with open(json_file, 'r') as f:
        param_data = json.load(f)
        P = param_data['camera']['P']
        d = param_data['camera']['distortion']
        return np.array(P).reshape([3,3]), np.array(d) # ok 

In [ ]:
def calibrate(dirnames, gc_fname_lists, proj_shape, chess_shape, chess_block_size, gc_step, black_thr, white_thr, camP, camD):
# proj_shape = (2160, 3840) chess_shape = (10, 7)    

    # (1) object points
    objps = np.zeros((chess_shape[0]*chess_shape[1], 3), np.float32) 
    # object points. 70행 3열 2차원 배열이 나타남. 
    objps[:, :2] = chess_block_size * \
        np.mgrid[0:chess_shape[0], 0:chess_shape[1]].T.reshape(-1, 2)
        # [[0,0], ..., [9,0], ..., [9,9]] 이렇게 바뀜. 
        # 모든 행 2열 전까지. 즉 열 2개만. 3열은 0만.
        # ok

    # (2) projector size -> gray code 
    print('Calibrating ...') # 계산중임다. 
    gc_height = int((proj_shape[0]-1)/gc_step)+1 # 2160. 
    gc_width = int((proj_shape[1]-1)/gc_step)+1 # 3840. 

##### 이 부분이 
    graycode = cv2.structured_light_GrayCodePattern.create(
        gc_width, gc_height) # 3840 * 2160 -> 이거로 50장 만들어내는 듯.
    graycode.setBlackThreshold(black_thr)  
    graycode.setWhiteThreshold(white_thr) 
##### 이해가 잘 안가요.  

    # (3) graycode_* size, patch_size 
    cam_shape = cv2.imread(gc_fname_lists[0][0], cv2.IMREAD_GRAYSCALE).shape 
    # cam_shape = cv2.imread(gc_fname_lists[0][0]).shape
    # 이미지 읽기. gc_fname_lists[0][0] = graycode_00
    # gc_fname_list 보니 맨 첫 원소는 [capture_0, graycode_00]임. 
    # cam_shape = 1600*2400 
    patch_size_half = int(np.ceil(cam_shape[1] / 180)) # np.ceil 하면 그 값들이 '올림'된다. cam_shape = (1600, 2400)임. 왜 이건지 모르겠다. 14
    # patch_size_half = int(np.ceil(cam_shape[1] / 270)) # 이렇게 하면 RMS가 5로 떨어지긴 하는데 카메라 내부 파라미터는 완전히 달라서 의미가 없을 듯.   
    print('  patch size :', patch_size_half * 2 + 1) # 29

    cam_corners_list = [] # 카메라 코너들을 모을 예정. 
    cam_objps_list = [] # 카메라 object들을 모을 예정. 
    cam_corners_list2 = [] # 보류
    _cam_corners_list = [] 

    proj_corners_list = [] # 프로젝터 코너들을 모을 예정
    proj_objps_list = [] # 프로젝터 object들을 모을 예정 
    
    # (4) dirnames = capture_*, gc_fname_lists = graycode_*
    for dname, gc_filenames in zip(dirnames, gc_fname_lists): # 폴더-파일  
        print('  checking \'' + dname + '\'') # dirname = capture_*
        if len(gc_filenames) != graycode.getNumberOfPatternImages() + 2:
            print('Error : invalid number of images in \'' + dname + '\'')
            return None 
            # graycode.getNumberOfPatternImages() = 48. 2를 더해서 50이 아닐 경우 에러내라. 

        imgs = []
        _imgpoints = []

        for fname in gc_filenames: # 각 graycode_*
            img = cv2.imread(fname)
            # img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE) # graycode_*를 흑백 모드에서 이미지 로드. 채널 하나구나.       
            # 앞에 _를 붙힌 변수는 image point들을 camera calibration과 동일하게 설정하려 임의로 추가한 코드임다. 
            # _gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _gray = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)

            if cam_shape != _gray.shape: # (원래) img.shape: # cam_shape = (1600, 2400, 3)
                print('Error : image size of \'' + fname + '\' is mismatch')
                return None # 흑백 모드에서 로드한 이미지가 (1600, 2400)의 사이즈를 갖지 않으면 에러내라.
                # gray scale로 하면 shape이 바뀜.  

            #imgs.append(img) # 빈 리스트에 img(graycode_*)를 연쇄적으로 추가함.  
            imgs.append(_gray)

        black_img = imgs.pop() #imgs.pop() # graycode_49를 black_img로 지정함. 대다수의 0을 가진 매트릭스. graycode_49는 프로젝터에서 검정색 화면만 쏜 것임. 
        white_img = imgs.pop() #imgs.pop() # graycode_48을 white_img로 지정함. 37~60을 가진 매트릭스. graycode_48은 프로젝터에서 하얀색 화면만 쏜 것임.
        # pop() 2회를 통해 imgs에는 48장의 이미지만이 남음.  
        # cv2_imshow(white_img)를 통해 black_img와 white_img를 확인할 수 있다. 

        # 원래 코드 
        res, cam_corners = cv2.findChessboardCorners(white_img, chess_shape) # white_img를 통해 체스보드판의 코너들을 찾아낸다. 
        
        if not res: # res는 코너점들을 찾았는지 알려줌. 
            print('Error : chessboard was not found in \'' +
                  gc_filenames[-2] + '\'')  
            return None

        # 수정한 코드 
        # _ret, _corners = cv2.findChessboardCorners(_gray, (10, 7), None)
        _ret, _corners = cv2.findChessboardCorners(white_img, (10, 7), None)

        if _ret == True:
            _corners2 = cv2.cornerSubPix(white_img, _corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            _imgpoints.append(_corners2)

        #_ChessboardCorners = cv2.drawChessboardCorners(img, (10, 7), _corners2, _ret) 
        #if dname == "./capture_00" :
        #    plt.figure(figsize=(30, 30))
        #    plt.imshow(_ChessboardCorners)
        #    plt.axis("off")
        #    plt.show()

        cam_objps_list.append(objps) # 카메라 object들을 모아놓음. 
        cam_corners_list.append(cam_corners) # 카메라 코너들을 모아놓음. 
        _cam_corners_list.append(_corners2) 

        # procam calibration을 본격화하려는 모양. 
        # 위에서 정의한 proj_corners_list = [], proj_objps_list = [] 이 두 변수와 아래 변수들이 뭐가 다른지 생각 좀 해야할 듯. 
        proj_objps = []
        proj_corners = []
        cam_corners2 = []

        # viz_proj_points = np.zeros(proj_shape, np.uint8)
        for corner, objp in zip(cam_corners, objps): # 코너들 - object들   
            c_x = int(round(corner[0][0])) # round 함수는 반올림. c_x는 코너(1).
            c_y = int(round(corner[0][1])) # c_y는 코너(2).
            src_points = []
            dst_points = []
            for dx in range(-patch_size_half, patch_size_half + 1): # -14~15. 29만큼, path_size만큼 for문이 돌아감. 
                for dy in range(-patch_size_half, patch_size_half + 1):
                    x = c_x + dx 
                    y = c_y + dy
                    # 최소 196개의 (x,y) 좌표를 가져와야 ok. 

                    # white_img[y, x]는 (y, x) 지점의 픽셀값을 보여줌. 
                    if int(white_img[y, x]) - int(black_img[y, x]) <= black_thr: # white_img, black_img의 (x, y) 지점에서 차이를 구할 때 그 픽셀값이 black_thr = 40보다 작으면 okay. 
                        continue # 그 차이가 40보다 작아야지 (x, y) 좌표를 가져올 수 있는 건데 capture_1의 경우 그 차이가 40보다 큰 것임. 

                    err, proj_pix = graycode.getProjPixel(imgs, x, y) # projector 관련.

                    if not err:
                        src_points.append((x, y)) # src_points에 (x, y) 값들이 기록되는 것임.   
                        dst_points.append(gc_step*np.array(proj_pix)) # 보류. 
            if len(src_points) < patch_size_half**2: # path_size_half**2 = 196. (x, y)가 모여서 196개는 나와야 하는데 (x, y)가 덜 모이면 에러가 발생하는 것임.  
                print(
                    '    Warning : corner', c_x, c_y,
                    'was skiped because decoded pixels were too few (check your images and threasholds)') # 디코드된 값이 너무 적다고.
                continue
            h_mat, inliers = cv2.findHomography(
                np.array(src_points), np.array(dst_points))
            point = h_mat@np.array([corner[0][0], corner[0][1], 1]).transpose()
            point_pix = point[0:2]/point[2]
            proj_objps.append(objp)
            proj_corners.append([point_pix])
            cam_corners2.append(corner)
            # viz_proj_points[int(round(point_pix[1])),
            #                 int(round(point_pix[0]))] = 255

        if len(proj_corners) < 3:
            print('Error : too few corners were found in \'' +
                  dname + '\' (less than 3)')
            return None
        proj_objps_list.append(np.float32(proj_objps))
        proj_corners_list.append(np.float32(proj_corners))
        cam_corners_list2.append(np.float32(cam_corners2))
        # cv2.imwrite('visualize_corners_projector_' +
        #             str(cnt) + '.png', viz_proj_points)
        # cnt += 1

    print('Initial solution of camera\'s intrinsic parameters')
    cam_rvecs = []
    cam_tvecs = []
    if(camP is None):
        ret, cam_int, cam_dist, cam_rvecs, cam_tvecs = cv2.calibrateCamera(
            cam_objps_list, _cam_corners_list, _gray.shape[::-1], None, None, None, None) # cam_shape -> _gray.shape[::-1]
        print('  RMS :', ret)
    else:
        for objp, corners in zip(cam_objps_list, _cam_corners_list):##
            ret, cam_rvec, cam_tvec = cv2.solvePnP(objp, corners, camP, camD) # 외부 파라미터 구하는 함수.  
            cam_rvecs.append(cam_rvec)
            cam_tvecs.append(cam_tvec)
            print('  RMS :', ret)
        cam_int = camP
        cam_dist = camD
    print('  Intrinsic parameters :')
    printNumpyWithIndent(cam_int, '    ')
    print('  Distortion parameters :')
    printNumpyWithIndent(cam_dist, '    ')
    cam_rotation_matrix = np.zeros(shape=(3, 3))
    cv2.Rodrigues(cam_rvecs[0], cam_rotation_matrix) 
    # Rodrigues로 표현된(3*1) R vectors를 3*3으로 표현하기 위함.
    # 체스보드를 바닥에 딱 붙힌 capture_00에 대해서 rotation vector를 구하겠음.  
    print('  cam_rvecs_capture_00 :')
    printNumpyWithIndent(cam_rotation_matrix, '    ')
    print('  cam_tvecs_capture_00 :')
    printNumpyWithIndent(cam_tvecs[0], '    ')
    print()

    print('Initial solution of projector\'s parameters')
    ret, proj_int, proj_dist, proj_rvecs, proj_tvecs = cv2.calibrateCamera(
        proj_objps_list, proj_corners_list, proj_shape, None, None, None, None)
    
    print('  RMS :', ret)
    print('  Intrinsic parameters :')
    printNumpyWithIndent(proj_int, '    ')
    print('  Distortion parameters :')
    printNumpyWithIndent(proj_dist, '    ')
    proj_rotation_matrix = np.zeros(shape=(3, 3))
    cv2.Rodrigues(proj_rvecs[0], proj_rotation_matrix) # Rodrigues로 표현된(3*1) R vectors를 3*3으로 표현하기 위함. 
    print('  proj_rvecs_capture_00 :')
    printNumpyWithIndent(proj_rotation_matrix, '    ')
    print('  proj_tvecs_capture_00 :')
    printNumpyWithIndent(proj_tvecs[0], '    ') 
    print()

    print('=== Result ===')
    ret, cam_int, cam_dist, proj_int, proj_dist, cam_proj_rmat, cam_proj_tvec, E, F = cv2.stereoCalibrate(
        proj_objps_list, cam_corners_list2, proj_corners_list, cam_int, cam_dist, proj_int, proj_dist, None)
    print('  RMS :', ret)
    print('  Camera intrinsic parameters :')
    printNumpyWithIndent(cam_int, '    ')
    print('  Camera distortion parameters :')
    printNumpyWithIndent(cam_dist, '    ')
    print('  Projector intrinsic parameters :')
    printNumpyWithIndent(proj_int, '    ')
    print('  Projector distortion parameters :')
    printNumpyWithIndent(proj_dist, '    ')
    print('  Rotation matrix / translation vector from camera to projector')
    print('  (they translate points from camera coord to projector coord) :')
    printNumpyWithIndent(cam_proj_rmat, '    ')
    printNumpyWithIndent(cam_proj_tvec, '    ')
    print()

    fs = cv2.FileStorage('calibration_result.xml', cv2.FILE_STORAGE_WRITE)
    fs.write('img_shape', cam_shape)
    fs.write('rms', ret)
    fs.write('cam_int', cam_int)
    fs.write('cam_dist', cam_dist)
    fs.write('proj_int', proj_int)
    fs.write('proj_dist', proj_dist)
    fs.write('roration', cam_proj_rmat)
    fs.write('translation', cam_proj_tvec)
    fs.release()



In [ ]:
path = Path('/content/gdrive/My Drive/danbi-project/real_data_200605')
#%cd /content/gdrive/My\ Drive/procam-calibration/sample_data
# !pwd # show me the current working directory

In [ ]:
cd /content/drive/"My Drive"/danbi-project/real_data_200605

/content/drive/My Drive/danbi-project/real_data_200605


In [ ]:
ls

calibration_result.xml  capture_1/  capture_3/   capture_5/
capture_0/              capture_2/  _capture_4/


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
proj_height = 2160 # 변경 불가
proj_width = 3840 # 변경 불가
chess_rows = 10 # 변경 불가
chess_cols = 7 # 변경 불가 
chess_block_size = 68 # 뭘로 바꿔도 똑같은 듯. 
graycode_step = 1
black_thr = 40
white_thr = 1
camera =  ""

main(proj_height, proj_width, chess_rows, chess_cols, chess_block_size,
    graycode_step, black_thr, white_thr,  camera)

Searching input files ...
 './capture_0' was found
 './capture_1' was found
 './capture_2' was found
 './capture_3' was found
 './capture_5' was found
Calibrating ...
  patch size : 29
  checking './capture_0'
  checking './capture_1'
  checking './capture_2'
  checking './capture_3'
  checking './capture_5'
Initial solution of camera's intrinsic parameters
  RMS : 1.5665573570371265
  Intrinsic parameters :
    [[1.92288454e+03 0.00000000e+00 1.18954599e+03]
     [0.00000000e+00 1.92031141e+03 9.45177448e+02]
     [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
  Distortion parameters :
    [[-0.27791696  0.40290588  0.01531431 -0.00324072 -0.38488884]]
  cam_rvecs_capture_00 :
    [[ 0.99765366  0.06804745  0.007531  ]
     [-0.06364915  0.8813591   0.46813985]
     [ 0.02521821 -0.46752078  0.8836223 ]]
  cam_tvecs_capture_00 :
    [[-401.69137375]
     [-305.18491107]
     [1096.26402925]]

Initial solution of projector's parameters
  RMS : 0.9076496005213202
  Intrinsic parameters

In [ ]:
import numpy as np
# 폴더 5개 돌린거 
# camera calibration 결과 총 정리. 
tvec_wc = np.array([-401.69137375, -305.18491107, 1096.26402925])
rvec_wc = np.array([[ 0.99765366,  0.06804745,  0.007531  ],
                    [-0.06364915,  0.8813591,   0.46813985],
                    [ 0.02521821, -0.46752078,  0.8836223 ]])
print("tvec_wc : \n", tvec_wc)
print("rvec_wc : \n", rvec_wc)
# Pc = R*Pw + T
# Pw = R.T*Pc - R.T*T

tvec_cw = -np.matmul(rvec_wc.T, tvec_wc)
rvec_cw = rvec_wc.T
print("tvec_cw : \n", tvec_cw)
print("rvec_cw : \n", rvec_cw)

tvec_cp = np.array([ -14.64331615, -633.36927611, 1055.26471553])
rvec_cp = np.array([[-0.99820069,  0.04397614, -0.0407612],
                   [-0.05771089, -0.88910474,  0.45405089],
                   [-0.01627358,  0.45558628,  0.89004287]])
print("tvec_cp : \n", tvec_cp)
print("rvec_cp : \n", rvec_cp)

tvec_pc = -np.matmul(rvec_cp.T, tvec_cp)
rvec_pc = rvec_cp.T
print("tvec_pc : \n", tvec_pc)
print("rvec_pc : \n", rvec_pc)

tvec_wp = np.array([ 326.22694335, 159.83415869, 1893.37003383])
rvec_wp = np.array([[-0.99983442, -0.0087632,  -0.01594788],
                   [ 0.00894139, -0.99989801, -0.01113672],
                   [-0.01584866, -0.01127747,  0.9998108 ]])
print("tvec_wp : \n", tvec_wp)
print("rvec_wp : \n", rvec_wp)

tvec_pw = -np.matmul(rvec_wp.T, tvec_wp)
rvec_pw = rvec_wp.T
print("tvec_pw : \n", tvec_pw)
print("rvec_pw : \n", rvec_pw)

tvec_wc : 
 [-401.69137375 -305.18491107 1096.26402925]
rvec_wc : 
 [[ 0.99765366  0.06804745  0.007531  ]
 [-0.06364915  0.8813591   0.46813985]
 [ 0.02521821 -0.46752078  0.8836223 ]]
tvec_cw : 
 [ 353.67829252  808.83778627 -822.78898671]
rvec_cw : 
 [[ 0.99765366 -0.06364915  0.02521821]
 [ 0.06804745  0.8813591  -0.46752078]
 [ 0.007531    0.46813985  0.8836223 ]]
tvec_cp : 
 [ -14.64331615 -633.36927611 1055.26471553]
rvec_cp : 
 [[-0.99820069  0.04397614 -0.0407612 ]
 [-0.05771089 -0.88910474  0.45405089]
 [-0.01627358  0.45558628  0.89004287]]
tvec_pc : 
 [  -33.99633814 -1043.2517952   -652.24583164]
rvec_pc : 
 [[-0.99820069 -0.05771089 -0.01627358]
 [ 0.04397614 -0.88910474  0.45558628]
 [-0.0407612   0.45405089  0.89004287]]
tvec_wp : 
 [ 326.22694335  159.83415869 1893.37003383]
rvec_wp : 
 [[-0.99983442 -0.0087632  -0.01594788]
 [ 0.00894139 -0.99989801 -0.01113672]
 [-0.01584866 -0.01127747  0.9998108 ]]
tvec_pw : 
 [  354.75116506   184.02907291 -1886.0291518 ]
rvec_pw 

In [ ]:
# 카메라 내부 파라미터 매트릭스. 
mtx = np.array([[1.92288454e+03, 0.00000000e+00, 1.18954599e+03],
               [0.00000000e+00, 1.92031141e+03, 9.45177448e+02],
               [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
# 카메라 왜곡 계수. 
dist = np.array([[-0.27791696,  0.40290588,  0.01531431, -0.00324072, -0.38488884]])

In [ ]:
# 프로젝터 내부 파라미터 매트릭스. 
mtx = np.array([[2.87597491e+03, 0.00000000e+00, 2.03138471e+03],
               [0.00000000e+00, 2.87363389e+03, 1.20204078e+03],
               [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
# 프로젝터 왜곡 계수. 
dist = np.array([[-1.23745111e-01, -8.26584913e-02,  7.51610155e-05,  8.28977054e-03,  4.43302944e-01]])

In [ ]:
#img = cv2.imread("/content/drive/My Drive/danbi-project/real_data_200605/capture_5/graycode_35.JPG")
img = cv2.imread("/content/drive/My Drive/_camera_1.jpg")
h, w = img.shape[:2]
newcameraMtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [ ]:
dst = cv2.undistort(img, mtx, dist, None, newcameraMtx)

x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite("projector_camera_1.jpg", dst)

True

In [ ]:
cp "/content/projector_camera_1.jpg" "/content/drive/My Drive"

(1)

Pc = Rwc * Pw + Twc

POc = (0, 0, 0)일 때, 

POwc = -Rwc.T * Twc

In [ ]:
# 명칭 바꾸었음. POwc : 앞에 거(world) 원점 기준으로, 뒤에 거(camera) 원점 좌표를 구함. 
import numpy as np

# 체스보드 코너점(left-top, world), 즉 월드 원점(world)을 기준으로 카메라(camera) 원점이 어디에 위치하는가. 
# Xw 방향으로 35cm, Yw 방향으로 133cm, Zw 방향으로 -153cm 이동한 지점에 카메라 원점이 위치함. 
POwc = np.matmul(-rvec_wc.T, tvec_wc)
POwc

array([ 353.67829252,  808.83778627, -822.78898671])

(2) 

Pc = Rwc * Pw + Twc

POw = (0, 0, 0)

POcw = Twc

(1), (2)와 동일한 방법으로 camera-projector, world-projector의 PO를 구하기. 

In [ ]:
# 카메라 원점을 기준으로 월드 원점이 어디에 위치하는가.
# Xc 방향으로 -32cm, Yc 방향으로 -58cm, Zc 방향으로 195cm 이동한 지점에 월드 원점이 위치함. 

POcw = tvec_wc # P0w = np.zeros([3, 1])
POcw

array([-401.69137375, -305.18491107, 1096.26402925])

In [ ]:
# 월드 원점을 기준으로 프로젝터 원점이 어디에 위치하는가.
# Xw 방향으로 38cm, Yw 방향으로 14cm, Zw 방향으로 -217cm 이동한 지점에 프로젝터 원점이 위치함.  
POwp = np.matmul(-rvec_wp.T, tvec_wp)
POwp

array([  354.75116506,   184.02907291, -1886.0291518 ])

In [ ]:
# 프로젝터 원점을 기준으로 월드 원점이 어디에 위치하는가.
# Xp 방향으로 40cm, Yw 방향으로 12cm, Zp 방향으로 216cm 이동한 지점에 월드 원점이 위치함.  
POpw = tvec_wp
POpw

array([ 326.22694335,  159.83415869, 1893.37003383])

In [ ]:
# 카메라 원점을 기준으로 프로젝터 원점이 어디에 위치하는가. 
# Xc 방향으로 0.2cm, Yc 방향으로 -132cm, Zc 방향으로 -10cm 이동한 지점에 프로젝터 원점이 위치함. 
POcp = np.matmul(-rvec_cp.T, tvec_cp)
POcp

array([  -33.99633814, -1043.2517952 ,  -652.24583164])

In [ ]:
# 프로젝터 원점을 기준으로 카메라 원점이 어디에 위치하는가.
# Xp 방향으로 3cm, Yp 방향으로 -118cm, Zp 방향으로 61cm 이동한 지점에 카메라 원점이 위치함.  
POpc = tvec_cp
POpc

array([ -14.64331615, -633.36927611, 1055.26471553])

(1) Pc = RwcPw + Twc

(2) Pp = RcpPc + Tcp

(3) : (1)의 Pc 식을 (2)에 대입함.

Pp = (Rcp)(RwcPw + Twc) + Tcp

Pp = (RcpRwc)Pw + RcpTwc + Tcp

(4) : world-camera, camera-projector의 관계식을 연립하여 구한 (3)의 식이 world-projector의 관계식(Pp = RwpPw + Twp)과 일치하는지 확인함. 

즉, Rwp가 (RcpRwc)와 일치하는지, Twp가 (RcpTwc + Tcp)와 일치하는지 보려고 한 것임. 


In [ ]:
# 명칭 유의. 연립을 통해 새로 구한 값은 대문자로 표시하였음

rvec_WP = np.matmul(rvec_cp, rvec_wc)
tvec_WP = np.matmul(rvec_cp, tvec_wc) + tvec_cp
print("rvec_wp \n", rvec_wp) 
print("rvec_WP \n", rvec_WP)
print("tvec_wp \n", tvec_wp)
print("tvec_WP \n", tvec_WP)

rvec_wp 
 [[-0.99983442 -0.0087632  -0.01594788]
 [ 0.00894139 -0.99989801 -0.01113672]
 [-0.01584866 -0.01127747  0.9998108 ]]
rvec_WP 
 [[-0.99968554 -0.01010953 -0.02294797]
 [ 0.01046563 -0.99982586 -0.01545049]
 [-0.02278779 -0.0156858   0.99961726]]
tvec_wp 
 [ 326.22694335  159.83415869 1893.37003383]
tvec_WP 
 [ 328.21939857  158.91369974 1898.48559676]


rvec_wp와 rvec_WP가 육안으로 보기에는 다소 차이가 있어 이들의 theta 값을 구하고자 하였음. (https://darkpgmr.tistory.com/99?category=460965 를 참고)

In [ ]:
import math
theta_wp = math.acos((np.trace(rvec_wp)-1)/2)
theta_WP = math.acos((np.trace(rvec_WP)-1)/2)
angle_wp = theta_wp*(180/3.14)
angle_WP = theta_WP*(180/3.14)
theta_wp, theta_WP, angle_wp, angle_WP

(3.136357191565369, 3.135510238586101, 179.79117658654982, 179.7426251418784)

이들의 theta 값을 이용해 각각의 회전행렬, Rt_wp와 Rt_WP를 구하였음. 

In [ ]:
theta = theta_wp
Rx_wp = np.array([[1, 0, 0],
                  [0, math.cos(theta), -math.sin(theta)],
                  [0, math.sin(theta), math.cos(theta)]])

Ry_wp = np.array([[math.cos(theta), 0, math.sin(theta)],
                  [0, 1, 0],
                  [-math.sin(theta), 0, math.cos(theta)]])

Rz_wp = np.array([[math.cos(theta), -math.sin(theta), 0],
                  [math.sin(theta), math.cos(theta), 0],
                  [0, 0, 1]])

Rt_wp = np.matmul(Ry_wp, Rx_wp)
Rt_wp = np.matmul(Rz_wp, Rt_wp)
print("Rt_wp \n", Rt_wp)

theta = theta_WP
Rx_WP = np.array([[1, 0, 0],
                  [0, math.cos(theta), -math.sin(theta)],
                  [0, math.sin(theta), math.cos(theta)]])

Ry_WP = np.array([[math.cos(theta), 0, math.sin(theta)],
                  [0, 1, 0],
                  [-math.sin(theta), 0, math.cos(theta)]])

Rz_WP = np.array([[math.cos(theta), -math.sin(theta), 0],
                  [math.sin(theta), math.cos(theta), 0],
                  [0, 0, 1]])

Rt_WP = np.matmul(Ry_WP, Rx_WP)
Rt_WP = np.matmul(Rz_WP, Rt_WP)
print("Rt_WP \n", Rt_WP)

Rt_wp 
 [[ 0.99997259  0.00520796  0.0052627 ]
 [-0.00523537  0.99997273  0.00520796]
 [-0.00523544 -0.00523537  0.99997259]]
Rt_WP 
 [[ 0.999963    0.00604527  0.00611915]
 [-0.00608226  0.99996323  0.00604527]
 [-0.00608238 -0.00608226  0.999963  ]]


In [ ]:
# rotation world to camera
theta_wc = math.acos((np.trace(rvec_wc)-1)/2)
theta = theta_wc

Rx_wc = np.array([[1, 0, 0],
                  [0, math.cos(theta), -math.sin(theta)],
                  [0, math.sin(theta), math.cos(theta)]])

Ry_wc = np.array([[math.cos(theta), 0, math.sin(theta)],
                  [0, 1, 0],
                  [-math.sin(theta), 0, math.cos(theta)]])

Rz_wc = np.array([[math.cos(theta), -math.sin(theta), 0],
                  [math.sin(theta), math.cos(theta), 0],
                  [0, 0, 1]])

Rt_wc = np.matmul(Ry_wc, Rx_wc)
Rt_wc = np.matmul(Rz_wc, Rt_wc)
print("angle_wc \n", theta_wc * (180/3.14))
print("Rt_wc \n", Rt_wc)

# rotation camera to projector
theta_cp = math.acos((np.trace(rvec_cp)-1)/2)
theta = theta_cp

Rx_cp = np.array([[1, 0, 0],
                  [0, math.cos(theta), -math.sin(theta)],
                  [0, math.sin(theta), math.cos(theta)]])

Ry_cp = np.array([[math.cos(theta), 0, math.sin(theta)],
                  [0, 1, 0],
                  [-math.sin(theta), 0, math.cos(theta)]])

Rz_cp = np.array([[math.cos(theta), -math.sin(theta), 0],
                  [math.sin(theta), math.cos(theta), 0],
                  [0, 0, 1]])

Rt_cp = np.matmul(Ry_cp, Rx_cp)
Rt_cp = np.matmul(Rz_cp, Rt_cp)
print("angle_cp \n", theta_cp * (180/3.14))
print("Rt_cp \n", Rt_cp)

angle_wc 
 24.163548645640567
Rt_wc 
 [[ 0.83259825 -0.2205854   0.50805731]
 [ 0.37333417  0.90109027 -0.2205854 ]
 [-0.40914759  0.37333417  0.83259825]]
angle_cp 
 179.1532603541071
Rt_cp 
 [[ 0.99973226  0.01609293  0.01662619]
 [-0.01636064  0.99973664  0.01609293]
 [-0.01636283 -0.01636064  0.99973226]]


In [ ]:
print(Rt_wp)
print(Rt_WP) # theta_WP를 통해 구한 회전행렬. 
print(np.matmul(Rt_cp, Rt_wc)) # rvec_cp, rvec_wc 각각의 theta를 구해 회전행렬을 구하고, 이들을 곱하여 Rt_wp와 일치하는지 확인함. 

[[ 0.99997259  0.00520796  0.0052627 ]
 [-0.00523537  0.99997273  0.00520796]
 [-0.00523544 -0.00523537  0.99997259]]
[[ 0.999963    0.00604527  0.00611915]
 [-0.00608226  0.99996323  0.00604527]
 [-0.00608238 -0.00608226  0.999963  ]]
[[ 0.83158081 -0.19981803  0.51821435]
 [ 0.35302963  0.91046992 -0.2154405 ]
 [-0.42876969  0.3621012   0.82767099]]


In [ ]:
# http://egloos.zum.com/EireneHue/v/982268
print(-math.asin(-0.4287))
print(0.4430 * (180/3.14))
print(-math.asin(-0.0052))
print(0.0052 * (180/3.14)) 


0.4430533513718603
25.394904458598724
0.005200023434951824
0.2980891719745223
